In [22]:
#Format data
import pandas as pd
fightList = pd.read_csv("ufc-master.csv").reset_index(drop=True)
#fightList.drop(columns=["Unnamed: 0"], inplace=True)


fightList[["RED_ELO_BEFORE","RED_ELO_PEAK", "BLUE_ELO_BEFORE", "BLUE_ELO_PEAK"]] = 1000

fightList.head()


,Winner,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,...,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds,RED_ELO_BEFORE,RED_ELO_PEAK,BLUE_ELO_BEFORE,BLUE_ELO_PEAK
0,Red,Brandon Royval,Tatsuro Taira,NaN,NaN,NaN,NaN,12/10/2024,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,1000,1000,1000,1000
1,Blue,Brad Tavares,JunYong Park,NaN,NaN,NaN,NaN,12/10/2024,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,1000,1000,1000,1000
2,Red,Chidi Njokuani,Jared Gooden,NaN,NaN,NaN,NaN,12/10/2024,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,1000,1000,1000,1000
3,Red,Grant Dawson,Rafa Garcia,-380.0,300.0,26.3158,300.0,12/10/2024,"Las Vegas, Nevada, USA",USA,...,-140.0,650.0,350.0,1600.0,900.0,800.0,1000,1000,1000,1000
4,Red,Daniel Rodriguez,Alex Morono,-245.0,200.0,40.8163,200.0,12/10/2024,"Las Vegas, Nevada, USA",USA,...,165.0,275.0,1100.0,1200.0,215.0,1000.0,1000,1000,1000,1000


In [23]:
def get_expected_score(elo_win, elo_loss):
    return 1 / (1 + 10 **((elo_loss - elo_win) / 400))

#This gives more points for a finish, even more if finish is in an earlier round
def get_k_factor(method, round, k = 100,):
    if method == "KO/TKO" or "Submission":
        match round:
            case 1: 
                return k * 1.2
            case 2: 
                return k * 1.18
            case _: 
                return k * 1.16
    if method == "Decision - Unanimous":
        return k * 1.14
    return k
    
def get_new_elo(win_elo, loss_elo, k_factor):
    exp_win = get_expected_score(win_elo, loss_elo)
    new_win_elo = win_elo + k_factor * (1-exp_win)
    new_loss_elo = loss_elo + k_factor * (0 - (1-exp_win))
    return round(new_win_elo, 2), round(new_loss_elo, 2)

In [24]:
elo_dict = {}
elo_peak_dict = {}

for i in range(len(fightList) - 1, -1, -1):
    #ufc generated
    #bout = fightList.loc[i, "BOUT"]
    #if " vs. " in bout:
    #    red_fighter, blue_fighter = bout.split(" vs. ")
    #else:
    #    red_fighter, blue_fighter = "Unknown_Red", "Unknown_Blue"

    #ufc-master
    red_fighter = fightList.loc[i, "RedFighter"]
    blue_fighter = fightList.loc[i, "BlueFighter"] 

    red_fighter = red_fighter.strip()
    blue_fighter = blue_fighter.strip()

    # Get current ELO (default 1000)
    red_elo_before = elo_dict.get(red_fighter, 1000)
    blue_elo_before = elo_dict.get(blue_fighter, 1000)

    # Get peak ELO going *into* the fight (i.e. max seen so far up to this point)
    red_peak_elo = max(red_elo_before, elo_peak_dict.get(red_fighter, 1000))
    blue_peak_elo = max(blue_elo_before, elo_peak_dict.get(blue_fighter, 1000))

    # Store peak ELO going *into* the fight
    fightList.loc[i, "RED_ELO_BEFORE"] = float(red_elo_before)
    fightList.loc[i, "BLUE_ELO_BEFORE"] = float(blue_elo_before)
    fightList.loc[i, "RED_ELO_PEAK"] = float(red_peak_elo)
    fightList.loc[i, "BLUE_ELO_PEAK"] = float(blue_peak_elo)

    # Fight outcome and K-factor
    win_method = fightList.loc[i, "Winner"] # METHOD
    rounds = fightList.loc[i, "FinishRound"]  #ROUND
    k_factor = get_k_factor(win_method, rounds)

    # ELO update based on outcome
    if fightList.loc[i, "Winner"] == "Red":    #OUTCOME
        red_elo_after, blue_elo_after = get_new_elo(red_elo_before, blue_elo_before, k_factor)
    elif fightList.loc[i, "Winner"] == "Blue":
        blue_elo_after, red_elo_after = get_new_elo(blue_elo_before, red_elo_before, k_factor)
    else:  # draw
        blue_elo_after, red_elo_after = get_new_elo(blue_elo_before, red_elo_before, k_factor / 2)

    # Update latest ELO
    elo_dict[red_fighter] = float(red_elo_after)
    elo_dict[blue_fighter] = float(blue_elo_after)

    # Update peak ELO if new peak achieved AFTER this fight
    elo_peak_dict[red_fighter] = max(elo_peak_dict.get(red_fighter, 1000), red_elo_after)
    elo_peak_dict[blue_fighter] = max(elo_peak_dict.get(blue_fighter, 1000), blue_elo_after)


C:\Users\craig\AppData\Local\Temp\ipykernel_15284\3170558528.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1009.09' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fightList.loc[i, "BLUE_ELO_BEFORE"] = float(blue_elo_before)
C:\Users\craig\AppData\Local\Temp\ipykernel_15284\3170558528.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1009.09' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fightList.loc[i, "BLUE_ELO_PEAK"] = float(blue_peak_elo)
C:\Users\craig\AppData\Local\Temp\ipykernel_15284\3170558528.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1067.59' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

In [25]:
elo_sorted = dict(sorted(elo_dict.items(), key=lambda item: item[1], reverse=True))
elo_peak_sorted = dict(sorted(elo_peak_dict.items(), key=lambda item: item[1], reverse=True))
elo_combined = {
    "key": list(elo_peak_sorted.keys()),
    "peak": [elo_peak_sorted[key] for key in elo_peak_sorted.keys()],
    "curr": [elo_dict[key] for key in elo_peak_sorted.keys()]
}

elo_df = pd.DataFrame(elo_combined)
elo_df.reset_index(drop=True, inplace=True)


elo_df = elo_df.sort_values(by=["peak"], ascending= False)
elo_df.to_csv("fighter-rankings.csv")
fightList.to_csv("ufc-master-elo.csv")
elo_df.head(20)



,key,peak,curr
0,Islam Makhachev,1697.98,1697.98
1,Kamaru Usman,1676.05,1440.51
2,Leon Edwards,1654.72,1579.19
3,Alexander Volkanovski,1627.41,1448.88
4,Belal Muhammad,1621.83,1621.83
5,Aljamain Sterling,1608.60,1528.07
6,Jon Jones,1607.64,1607.64
7,Khabib Nurmagomedov,1594.34,1594.34
8,Charles Oliveira,1588.50,1477.29
9,Israel Adesanya,1568.60,1370.21
